# SQL Avancé

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Révisions

\- Schémas

\- Création de table

> \- Clé primaire
> 
> \- Clé étrangère
> 
> \- Unicité

## Création de base

In [40]:
-- On supprime et on re-créé tout
IF (EXISTS (SELECT [name] FROM [master].[sys].[databases] WHERE [name] = 'Kiosque'))
    DROP DATABASE [Kiosque]

CREATE DATABASE [Kiosque]

GO

USE [Kiosque]

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.609

## Schémas

\- ref

\- data

Un schéma -\> regroupement d'éléments SQL (Tables, Vuews, ..) pour faciliter la gestion de permissions

In [41]:
CREATE SCHEMA [data]

GO

CREATE SCHEMA [ref]

GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.013

## Création de Tables

In [42]:
CREATE TABLE [data].[Utilisateur_UTL] (
    [Id] UNIQUEIDENTIFIER NOT NULL PRIMARY KEY DEFAULT NEWID(),
    [Login] NVARCHAR(50) NOT NULL UNIQUE,
    [Email] NVARCHAR(255) NOT NULL UNIQUE,
    [Actif] BIT NOT NULL DEFAULT 0
)

Commands completed successfully.

Total execution time: 00:00:00.012

In [43]:
CREATE TABLE [ref].[Role_ROL] (
    [Id] TINYINT NOT NULL PRIMARY KEY IDENTITY(1, 1),
    [Key] CHAR(10) NOT NULL,
    [Description] NVARCHAR(100) NOT NULL
)

Commands completed successfully.

Total execution time: 00:00:00.008

In [44]:
CREATE TABLE [data].[RoleUtilisateur_RUL] (
    [ROL_Id] TINYINT NOT NULL,
    [UTL_Id] UNIQUEIDENTIFIER NOT NULL,
    CONSTRAINT [pk_rol_utl] PRIMARY KEY ([ROL_Id], [UTL_Id]),
    CONSTRAINT [fk_RUL_ROL] FOREIGN KEY ([ROL_Id]) REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_RUL_UTL] FOREIGN KEY ([UTL_Id]) REFERENCES [data].[Utilisateur_UTL]([Id])
)

Commands completed successfully.

Total execution time: 00:00:00.010

## Modification de tables

In [45]:
ALTER TABLE [data].[Utilisateur_UTL]
    ADD [MotDePasse] NVARCHAR(512) NULL

Commands completed successfully.

Total execution time: 00:00:00.006

In [46]:
CREATE TABLE [ref].[Permission_PER] (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Nom] NVARCHAR(25) NOT NULL,
    [Description] NVARCHAR(256) NULL
)

Commands completed successfully.

Total execution time: 00:00:00.008

In [47]:
ALTER TABLE [ref].[Permission_PER]
    ADD CONSTRAINT [PK_PER] PRIMARY KEY ([Id])

Commands completed successfully.

Total execution time: 00:00:00.039

## Opréations CRUD

Create, Read, Update, Delete

  

Create -\>  INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [48]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('admin', N'Administrateur'),
    ('guest', N'Visiteur'),
    ('market', N'Marketing')

(3 rows affected)

Total execution time: 00:00:00.010

In [49]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('guest', N'Autre visiteur')

(1 row affected)

Total execution time: 00:00:00.008

In [50]:
SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

(4 rows affected)

Total execution time: 00:00:00.025

Id,Key,Description
1,admin,Administrateur
2,guest,Visiteur
3,market,Marketing
4,guest,Autre visiteur


In [51]:
DELETE FROM [ref].[Role_ROL]
WHERE [Id] IN (
    SELECT [Id]FROM [ref].[Role_ROL]
    WHERE [Description] = N'Autre visiteur'
)

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.027

Id,Key,Description
1,admin,Administrateur
2,guest,Visiteur
3,market,Marketing


In [52]:
ALTER TABLE [ref].[Role_ROL]
    ADD [ROL_Key] AS LOWER([Key]),
    CONSTRAINT [uq_key] UNIQUE([ROL_Key])

Commands completed successfully.

Total execution time: 00:00:00.014

In [53]:
--INSERT INTO [ref].[Role_ROL] ([Key], [Description])
--VALUES
--    ('guest', N'Autre visiteur')

Commands completed successfully.

Total execution time: 00:00:00.002

In [54]:
UPDATE [ref].[Role_ROL]
SET [Description] = CONCAT(N'Utilisateur ', LOWER([Description]))

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.032

Id,Key,Description
1,admin,Utilisateur administrateur
2,guest,Utilisateur visiteur
3,market,Utilisateur marketing


In [55]:
INSERT INTO [data].[Utilisateur_UTL] ([Login], [Email], [Actif])
VALUES
    (N'alice', N'alice@u-picardie.fr', 1),
    (N'bob', N'bob@u-picardie.fr', 1),
    (N'carl', N'carl@u-picardie.fr', 0)

GO

DECLARE @UtlId UNIQUEIDENTIFIER
DECLARE @RolId TINYINT

SELECT TOP 1 @UtlId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'alice'

SELECT TOP 1 @RolId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'admin'

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @UtlId AS [UTL_Id], @RolId AS [ROL_Id]


(3 rows affected)

(1 row affected)

Total execution time: 00:00:00.025

Pour la prochaine séance :

\- Ajouter le rôle Marketing à Alice

\- Ajouter le rôle Marketing à Bob

\- Ajouter le rôle Guest à Carl

In [56]:

DECLARE @AliceId UNIQUEIDENTIFIER
DECLARE @BobId UNIQUEIDENTIFIER
DECLARE @CarlId UNIQUEIDENTIFIER
DECLARE @MarketId TINYINT
DECLARE @GuestId TINYINT

SELECT TOP 1 @AliceId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'alice'

SELECT TOP 1 @BobId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'bob'

SELECT TOP 1 @CarlId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'carl'

SELECT TOP 1 @MarketId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'market'

SELECT TOP 1 @GuestId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'guest'

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @AliceId AS [UTL_Id], @MarketId AS [ROL_Id]

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @BobId AS [UTL_Id], @MarketId AS [ROL_Id]

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @CarlId AS [UTL_Id], @GuestId AS [ROL_Id]


(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.021

Créer une table de liaison PermissionRole\_PRO entre Role et permission (schéma ref)

Ajouter trois permissions : 

\- CREATE\_CONTENT

\- READ\_CONTENT

\- EDIT\_CONTENT

Les liées au rôle admin

Lier uniquement READ\_CONTENT et EDIT\_CONTENT au rôle marketing

Lier READ\_CONTENT au rôle guest

In [57]:
CREATE TABLE [ref].[PermissionRole_PRO] (
    [ROL_Id] TINYINT NOT NULL,
    [PER_Id] INT NOT NULL,
    CONSTRAINT [pk_rol_per] PRIMARY KEY ([ROL_Id], [PER_Id]),
    CONSTRAINT [fk_PRO_ROL] FOREIGN KEY ([ROL_Id]) REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_PRO_PER] FOREIGN KEY ([PER_Id]) REFERENCES [ref].[Permission_PER]([Id])
)

Commands completed successfully.

Total execution time: 00:00:00.011

In [58]:
INSERT INTO [ref].[Permission_PER] ([Nom], [Description])
VALUES
    (N'CREATE_CONTENT', N'Permission de créer du contenu'),
    (N'READ_CONTENT', N'Permission de consulter le contenu'),
    (N'EDIT_CONTENT', N'Permission d''éditer du contenu')

(3 rows affected)

Total execution time: 00:00:00.009

In [59]:
DECLARE @AdminId TINYINT
DECLARE @MarketId TINYINT
DECLARE @GuestId TINYINT

SELECT TOP 1 @AdminId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'admin'
SELECT TOP 1 @MarketId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'market'
SELECT TOP 1 @GuestId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'guest'

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @AdminId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'CREATE_CONTENT', N'READ_CONTENT', N'EDIT_CONTENT')

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @MarketId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'READ_CONTENT', N'EDIT_CONTENT')

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @GuestId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'READ_CONTENT')

(3 rows affected)

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.035

![Jointures SQL](https://i.pinimg.com/originals/78/06/27/78062746cee62bdc112aad0582ae49c4.jpg)

Récupérer les utilsateurs avec la permission READ\_CONTENT

Puis Récupérer les utlisateurs avec la permission EDIT\_CONTENT

In [60]:
SELECT DISTINCT [UTL].[Login]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]

INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]

INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]

WHERE [PER].[Nom] = N'READ_CONTENT'

(3 rows affected)

Total execution time: 00:00:00.045

Login
alice
bob
carl


In [61]:
SELECT DISTINCT [UTL].[Login]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)
INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]
INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]
INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]
INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]
WHERE [PER].[Nom] = N'EDIT_CONTENT'

(2 rows affected)

Total execution time: 00:00:00.024

Login
alice
bob


## Vues et Index

In [62]:
CREATE OR ALTER VIEW [data].[view_PermissionUtilisateur]
AS
SELECT DISTINCT [UTL].[Id] AS [UTL_Id], [UTL].[Login]
              , [PER].[Id] AS [PER_Id], [PER].[Nom] AS [Permission]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]

INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]

INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]

WHERE [UTL].[Actif] = 1

Commands completed successfully.

Total execution time: 00:00:00.009

In [63]:
SELECT [UTL_Id], [Login], [PER_Id], [Permission] FROM [data].[view_PermissionUtilisateur]

(5 rows affected)

Total execution time: 00:00:00.032

UTL_Id,Login,PER_Id,Permission
dde545ce-3ee8-486a-b09f-4144ddef0fe7,alice,1,CREATE_CONTENT
dde545ce-3ee8-486a-b09f-4144ddef0fe7,alice,2,READ_CONTENT
dde545ce-3ee8-486a-b09f-4144ddef0fe7,alice,3,EDIT_CONTENT
6e72c1cd-3d99-4fb3-a79c-479810064d66,bob,2,READ_CONTENT
6e72c1cd-3d99-4fb3-a79c-479810064d66,bob,3,EDIT_CONTENT


In [64]:
CREATE INDEX [IDX_UtlId_By_Login]
ON [data].[Utilisateur_UTL] ([Login], [Id])

Commands completed successfully.

Total execution time: 00:00:00.009

In [65]:
CREATE OR ALTER VIEW [data].[view_RoleUtilisateur]
WITH SCHEMABINDING
AS
SELECT [UTL].[Id] AS [UTL_Id], [UTL].[Login]
              , [ROL].[Id] AS [ROL_Id], [ROL].[Key] AS [Role]
    FROM [data].[Utilisateur_UTL] AS [UTL]

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL]
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL]
    ON [ROL].[Id] = [RUL].[ROL_Id]

WHERE [UTL].[Actif] = 1

Commands completed successfully.

Total execution time: 00:00:00.008

In [66]:
CREATE UNIQUE CLUSTERED INDEX [IDX_RoleUtilisateur_By_UtlId]
ON [data].[view_RoleUtilisateur] ([UTL_Id], [Role])

Commands completed successfully.

Total execution time: 00:00:00.017

In [67]:
CREATE UNIQUE INDEX [IDX_RoleUtilisateur_By_UtlLogin]
ON [data].[view_RoleUtilisateur] ([Login], [Role])

Commands completed successfully.

Total execution time: 00:00:00.010

# Fonctions

2 types :

\- Natives

\- UDF (User Defined Functions)

In [68]:
SELECT TOP 1
    GETDATE() AS [Date],
    DATEADD(DAY, 2, GETDATE()) AS [Après demain],
    DATEDIFF(DAY, DATEADD(MONTH, 2, GETDATE()), GETDATE()) AS [Diff deux mois],
    CONCAT([Login], '@', 'u-picardie.fr') AS [Email],
    COALESCE(null, 'Test') AS [Coalesce]
FROM [data].[Utilisateur_UTL]

(1 row affected)

Total execution time: 00:00:00.023

Date,Après demain,Diff deux mois,Email,Coalesce
2021-11-09 18:03:30.517,2021-11-11 18:03:30.517,-61,alice@u-picardie.fr,Test


In [69]:
CREATE FUNCTION [data].[fun_CountRolesForLogin](@Login NVARCHAR(50))
RETURNS INT
AS
BEGIN
    DECLARE @Retour INT

    SELECT @Retour = COUNT(*)
    FROM [data].[view_RoleUtilisateur]
    WHERE [Login] = @Login

    RETURN @Retour
END

Commands completed successfully.

Total execution time: 00:00:00.009

In [70]:
SELECT [data].[fun_CountRolesForLogin](N'alice')

(1 row affected)

Total execution time: 00:00:00.017

(No column name)
2


In [71]:
CREATE FUNCTION [data].[fun_GetRolesForLogin](@Login NVARCHAR(50))
RETURNS TABLE
AS
    RETURN SELECT [ROL_Id] AS [Id], [Role]
    FROM [data].[view_RoleUtilisateur]
    WHERE [Login]= @Login

Commands completed successfully.

Total execution time: 00:00:00.009

In [72]:
SELECT [Id], [Role] FROM [data].[fun_GetRolesForLogin](N'alice')

(2 rows affected)

Total execution time: 00:00:00.012

Id,Role
1,admin
3,market


# Projet

- Groupes de 1 à 5
- Rendu
    - Dictionnaire de données (justifier vos architecturaux)
    - 1 fichier de création de la BDD (SQL ou Notebook)
    - 1 fichier de remplissage de votre BDD (SQL ou Notebook)
    - 1 schéma graphique de votre BDD
- Logiciel de gestion de magazines :
    - Permissions utilisateurs
    - Magazines (1 éditeur, N distributeurs)
    - Editeurs (N magazines)
    - Distributeurs (N magazines)
    - Formats (papier A4, Papier autre, numérique)
- Ne pas en faire plus que demandé

## Contraintes

- 1 vue doit sortir les noms de magazines par éditeurs
- Les différentes tables doivent avoir des index pertinents (ex: Deux magazines ne peuvent pas avoir le même nom car ce sont des marques déposées)
- Une fonction doit retourner l'ensemble des distributeurs et éditeur d'un magazine